__ProPythia__

This file is a simulation for the allergenic dataset;

I intend to run propythia here so that in the future I can make comparisons with the results obtained with omnia.

i will do this simulation with the antioxidant data, where the data in unbalanced

# imports

In [0]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

from scipy.stats import loguniform
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, matthews_corrcoef,f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import VarianceThreshold


from propythia.protein.descriptors import ProteinDescritors
from propythia.protein.encoding import Encoding
from propythia.ml.shallow_ml import ShallowML
from propythia.protein.sequence import ReadSequence

from propythia.ml.deep_ml import DeepML

from propythia.feature_selection import FeatureSelection

from propythia.preprocess import Preprocess

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from itertools import product
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# nead to read the data : in our case antioxidant and allergenic data

# read_seqs = ReadSequence()


In [0]:
read_seqs = ReadSequence()
x_train_tabular = read_seqs.par_preprocessing(dataset= x_train, col = 'sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')
x_test_tabular = read_seqs.par_preprocessing(dataset= x_test, col = 'sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')

# Obtain all features

In [0]:
x_train_tabular= ProteinDescritors(dataset= x_train_tabular ,  col= 'sequence')
x_test_tabular= ProteinDescritors(dataset= x_test_tabular ,  col= 'sequence')

In [0]:
x_train_tabular=x_train_tabular.get_adaptable([17,21,29])
x_test_tabular=x_test_tabular.get_adaptable([17,21,29])

In [0]:
print(x_train_tabular.shape)

In [0]:
print(x_test_tabular.shape)

In [0]:
x_train_tabular = x_train_tabular.loc[:, ~x_train_tabular.columns.isin(["sequence"])]
x_test_tabular = x_test_tabular.loc[:, ~x_test_tabular.columns.isin(["sequence"])]

# scale

In [0]:
from sklearn.preprocessing import MinMaxScaler

scaler_train = MinMaxScaler()

x_train_tabular_scale = scaler_train.fit_transform(x_train_tabular)

scaler_test = MinMaxScaler()

x_test_tabular_scale = scaler_test.fit_transform(x_test_tabular)

x_train_tabular = pd.DataFrame(x_train_tabular_scale, columns=x_train_tabular.columns)
x_test_tabular = pd.DataFrame(x_test_tabular_scale, columns=x_test_tabular.columns)


In [0]:
print('train_x_tabular', x_train_tabular.shape)
print('test_x_tabular', x_test_tabular.shape)
print('train_y_tabular', y_train_tabular.shape)
print('test_y_tabular', y_test_tabular.shape)

In [0]:
x_train_tabular.to_csv('x_train_tabular.csv', index=False)  # Salvar o conjunto de treino de características

x_test_tabular.to_csv('x_test_tabular.csv', index=False)    # Salvar o conjunto de teste de características


# ShallowML

is only for the descriptores features because the tabular format of the data

In [0]:
import joblib
def train_multiple_models(X_train, X_test, y_train, y_test,model_names=['svc', 'linear_svc', 'rf', 'gboosting','knn', 'sgd', 'lr', 'nn']):
    """
    Train multiple models and return the results in a DataFrame.
    :param model_names: List with the names of the models to train.
    :param X_train: Training set features.
    :param X_test: Test set features.
    :param y_train: Training set labels.
    :param y_test: Test set labels.
    :param X: DataFrame with the features.
    :return: DataFrame with the results.
    """
    results = []
    ml = ShallowML(X_train, X_test, y_train, y_test, report_name=None, columns_names=None)
    for model_name in model_names:
        if model_name == 'svc':
            model_params = {'clf__C':[0.01, 1.0,10], 'clf__kernel': ['rbf','linear'], 'clf__gamma': ['auto', 'scale'], 'clf__class_weight': [None,'balanced']}
        elif model_name == 'linear_svc':
            model_params={'clf__C':[0.01, 1.0,10], 'clf__penalty': ['l2'], 'clf__class_weight': [None,'balanced']}
        elif model_name == 'rf':
            model_params = {'clf__n_estimators': [10,100,500], 'clf__max_features': [ 'sqrt', 'log2'],'clf__criterion':['gini','entropy'] , 'clf__class_weight': [None,'balanced']}
        elif model_name == 'gboosting':
            model_params = {'clf__n_estimators': [10, 100, 500], 'clf__max_depth': [1, 3, 5, 10],'clf__max_features': [0.6, 0.9],'clf__learning_rate':[0.1,1]}
        elif model_name == 'knn':
            model_params = {'clf__n_neighbors': [2,5,10,15],'clf__weights':['uniform', 'distance'], 'clf__leaf_size': [15, 30, 60]}
        elif model_name == 'sgd':
            model_params = {'clf__loss': ['hinge', 'log', 'modified_huber', 'perceptron'],'clf__alpha': [0.00001, 0.0001, 0.001, 0.01],'clf__early_stopping': [True],'clf__validation_fraction': [0.2],'clf__n_iter_no_change': [5,10],'clf__class_weight':[None,'balanced'] }
        elif model_name== 'lr':
            model_params = {'clf__C': [0.01, 0.1, 1.0, 10.0], 'clf__solver': ['liblinear', 'lbfgs', 'sag'], 'clf__class_weight': [None,'balanced']}
        elif model_name== 'nn':
            model_params = {'clf__hidden_layer_sizes':[(50,),(100,),(200,)],'clf__activation': ['logistic', 'tanh', 'relu'],'clf__solver':['adam','sgd'],'clf__alpha': [0.00001, 0.0001, 0.001],'clf__learning_rate_init': [0.0001, 0.001, 0.01]}
        else:
            raise ValueError(f"Model {model_name} not implemented.")
        # Call the function train best model to train the model
        best_classifier = ml.train_best_model(model_name, score =make_scorer(f1_score), optType='randomizedSearch',param_grid=model_params,cv=5,n_iter=50)
        scores, report, cm, cm2 = ml.score_testset(best_classifier)
        with open(f'scores_allergenic_data_shallow_{model_name}.txt', 'w') as f:
            for metric, value in scores.items():
                f.write(f"{metric}: {value}\n")
        
        y_pred = best_classifier.predict(ml.x_test)

        conf_mat = confusion_matrix(ml.y_test, y_pred)

        np.savetxt(f'confusion_matrix_allergenic_data_shallow_{model_name}.csv', conf_mat, delimiter=",")
        joblib.dump(best_classifier, f'best_classifier_data_tabular_shallow_{model_name}.h5')        

        results.append({
            'model_name': model_name,
            'best_params': best_classifier,
            'scores': scores
        })
    
    results_df=pd.DataFrame(results)
        
    return results_df,best_classifier
results_df = train_multiple_models(x_train_tabular, x_test_tabular, y_train_tabular, y_test_tabular, model_names=['svc', 'linear_svc', 'rf', 'gboosting','knn', 'sgd', 'lr', 'nn'])


# AutoEncoder

In [0]:
label_counts = y_test_tabular['label'].value_counts()  # Substitua 'label_column_name' pelo nome da coluna de rótulos, caso necessário

print("Contagem de rótulos no y_test:")
print(label_counts)

In [0]:
#autoencodermlp simultaneously and propythia
def create_autoencoder_mlp_model(latent_dim=12, num_layers=1, input_dim=100,neurons_per_layer=64, num_layers_class=1, neurons_mlp=32, num_classes=2, drop=0.3, activation="relu", last_layers_activations="sigmoid"):
    model = tf.keras.models.Sequential()

    # Encoder layers
    model.add(tf.keras.layers.Dense(input_dim, activation=activation))
    model.add(tf.keras.layers.Dropout(drop))
    for i in range(num_layers):
        current_neurons_per_layer= neurons_per_layer // (2**i)
        print(current_neurons_per_layer)
        model.add(tf.keras.layers.Dense(current_neurons_per_layer, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(latent_dim, activation=activation))
    model.add(tf.keras.layers.Dropout(drop))
    print(current_neurons_per_layer)
    # Decoder layers
    for i in reversed(range(num_layers)):
        current_neurons_per_layer= current_neurons_per_layer if i == 0 else current_neurons_per_layer *2
        print(current_neurons_per_layer)
        model.add(tf.keras.layers.Dense(current_neurons_per_layer, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(input_dim, activation=activation))

    # MLP layers
    for i in range(num_layers_class):
        current_neurons_mlp= neurons_mlp if i == 0 else neurons_mlp // (2**i)
        model.add(tf.keras.layers.Dense(current_neurons_mlp, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(num_classes, activation=last_layers_activations))

    model.compile(loss=['mse', 'binary_crossentropy'],loss_weights=[1, 1], optimizer='rmsprop', metrics=['accuracy'])
    return model

dl=DeepML(x_train_tabular, y_train_tabular, x_test_tabular, y_test_tabular, number_classes=2, problem_type='binary',
        x_dval=None, y_dval=None, epochs=100, batch_size=32,
        path='', report_name=None, verbose=1,
        early_stopping_patience=15, reduce_lr_patience=10, reduce_lr_factor=0.2, reduce_lr_min=0.00001,
                )
model = KerasClassifier(build_fn=create_autoencoder_mlp_model, verbose=0)

In [0]:
def generate_param_grid(num_layers:list, num_layers_class:list):
    param_grid = {
        'latent_dim': [12,6],
        'num_layers': num_layers,
        'input_dim': [3543],
        'neurons_per_layer': [64,128],
        'num_layers_class': num_layers_class,
        'neurons_mlp': [20,32],
        'num_classes': [2],
        'drop': [0.1,0.3],
        'activation': ['relu'],
        'last_layers_activations': ['sigmoid']
    }
    return param_grid

num_layers = [2,3]
num_layers_class = [2,3]
param_grid = generate_param_grid(num_layers, num_layers_class)

best_classifier_autoencoder=dl.get_opt_params(param_grid,model,scoring=make_scorer(f1_score),optType='randomizedSearch',cv=5,n_iter_search=50)
dl.save_model(path='best_data_tabular_autoencoder.h5', model=best_classifier_autoencoder)

In [0]:
scores, report, cm, cm2 = dl.score_testset(best_classifier_autoencoder)
with open('scores_data_tabular_autoencoder.txt', 'w') as f:
    for metric, value in scores.items():
        f.write(f"{metric}: {value}\n")


In [0]:
from sklearn.metrics import confusion_matrix


y_pred = best_classifier_autoencoder.predict(dl.x_test)

conf_mat = confusion_matrix(dl.y_test, y_pred)

np.savetxt('confusion_matrix_data_tabular_autoencoder.log.csv', conf_mat, delimiter=",")                        
